In [1]:
import numpy as np
import torch
import scipy
import sklearn.model_selection
import tensorflow as tf
import plotly

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cpu device


#  Load dataset 

In [3]:
mat_file = scipy.io.loadmat('ex7data.mat')

In [4]:
X, y = mat_file['X'].astype('float32'), np.ravel(mat_file['y'])

In [5]:
# Modify label 1-10 into 0-9
y[y == 10] = 0

In [6]:
X_new, X_test, y_new, y_test =  sklearn.model_selection.train_test_split(X, y, test_size= 0.3, train_size= 0.7,
                                                random_state= 1, shuffle=True, stratify= y)

X_train, X_valid, y_train, y_valid = sklearn.model_selection.train_test_split(X_new, y_new, test_size= 0.1, 
                                                train_size= 0.9, shuffle=True, stratify= y_new)

# Define

In [7]:
# Dataset class
class GetDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.data = data.astype(np.float32)
        self.label = label.astype(np.int64)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.label[idx]
        return image, label

In [8]:
class NeuralNetWork(torch.nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.linear_sigmoid_stack = torch.nn.Sequential(
            torch.nn.Linear(in_features=input_size, out_features=50),
            torch.nn.Sigmoid(),
            torch.nn.Linear(in_features=50, out_features=50),
            torch.nn.Sigmoid(),
            torch.nn.Linear(in_features=50, out_features=10)
        )
    def forward(self, X):
        logits = self.linear_sigmoid_stack(X)
        return logits

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction and loss
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss.item()

def valid_loop(dataloader, model, loss_fn, device):
    model.eval()
    num_batches = len(dataloader)
    test_loss, correct = 0, 0 

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            test_loss += loss_fn(y_pred, y).item()
            correct += (y_pred.argmax(1) == y).type(torch.float32).sum().item()

    test_loss /= num_batches
    correct /= len(dataloader.dataset)
    return test_loss, correct

In [10]:
# Create datasets and data loaders
train_data = GetDataset(X_train, y_train)
valid_data = GetDataset(X_valid, y_valid)
test_data = GetDataset(X_test, y_test)

train_dataloader = torch.utils.data.DataLoader(dataset=train_data, batch_size=200, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_data, batch_size=20, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(dataset=test_data, batch_size=200, shuffle=True)

In [11]:
input_size = X_train.shape[1]
epochs = 500
loss_fn = torch.nn.CrossEntropyLoss()
model = NeuralNetWork(input_size=input_size).to(device)

In [12]:
# Function to reset the model weights
def reset_model_weights(model):
    for layer in model.children():
        if hasattr(layer, 'reset_parameters'):
            layer.reset_parameters()

# So sánh giữa SGD có và không có momentum, điều chỉnh hệ số momentum

## Không có momentum

In [13]:
# Loss and optimizer
model = NeuralNetWork(input_size=input_size).to(device)
reset_model_weights(model)
lr = 0.11
optim_not_momentum = torch.optim.SGD(model.parameters(), lr=lr)

In [14]:
# Training
loss_train = []
loss_valid = []

for epoch in range(epochs):
    train_loss = train_loop(train_dataloader, model, loss_fn, optim_not_momentum, device)
    valid_loss, valid_accuracy = valid_loop(valid_dataloader, model, loss_fn, device)
    loss_train.append(train_loss)
    loss_valid.append(valid_loss)
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# Testing
test_loss, test_accuracy = valid_loop(test_dataloader, model, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 100, Train Loss: 0.9448, Valid Loss: 0.8743, Valid Accuracy: 0.7657
Epoch 200, Train Loss: 0.4094, Valid Loss: 0.4309, Valid Accuracy: 0.8857
Epoch 300, Train Loss: 0.2885, Valid Loss: 0.3609, Valid Accuracy: 0.9029
Epoch 400, Train Loss: 0.1749, Valid Loss: 0.3277, Valid Accuracy: 0.9057
Epoch 500, Train Loss: 0.1241, Valid Loss: 0.3186, Valid Accuracy: 0.9114
Test Loss: 0.3557, Test Accuracy: 0.9073


In [15]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x= np.arange(len(loss_train)), y= loss_train, name= 'Train'))
fig.add_trace(go.Scatter(x= np.arange(len(loss_valid)), y= loss_valid, name= 'Valid'))
fig.show()

## Có momentum

In [16]:
# Experiment 2: SGD with momentum
model = NeuralNetWork(input_size=input_size).to(device)
reset_model_weights(model)
lr = 0.1
momentum_value = 0.9
optimizer_sgd_momentum = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum_value)


In [17]:
# Training
loss_train = []
loss_valid = []

for epoch in range(epochs):
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer_sgd_momentum, device)
    valid_loss, valid_accuracy = valid_loop(valid_dataloader, model, loss_fn, device)
    loss_train.append(train_loss)
    loss_valid.append(valid_loss)
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# Testing
test_loss, test_accuracy = valid_loop(test_dataloader, model, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 100, Train Loss: 0.0382, Valid Loss: 0.3536, Valid Accuracy: 0.9200
Epoch 200, Train Loss: 0.0062, Valid Loss: 0.3621, Valid Accuracy: 0.9257
Epoch 300, Train Loss: 0.0045, Valid Loss: 0.4512, Valid Accuracy: 0.9343
Epoch 400, Train Loss: 0.0014, Valid Loss: 0.3928, Valid Accuracy: 0.9371
Epoch 500, Train Loss: 0.0016, Valid Loss: 0.4081, Valid Accuracy: 0.9400
Test Loss: 0.4145, Test Accuracy: 0.9200


In [ ]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x= np.arange(len(loss_train)), y= loss_train, name= 'Train'))
fig.add_trace(go.Scatter(x= np.arange(len(loss_valid)), y= loss_valid, name= 'Valid'))
fig.show()

### Kết luận: có momentum giúp mô hình hội tụ nhanh hơn, độ chính xác cao hơn so với khi không có momentum

# So sánh giữa momentum và nesterov momentum

In [19]:
# Loss and optimizer
model = NeuralNetWork(input_size=input_size).to(device)
reset_model_weights(model)
lr = 0.1
momentum_value = 0.9
optimizer_nesterov = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum_value, nesterov=True)


In [20]:
# Training
loss_train = []
loss_valid = []

for epoch in range(epochs):
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer_nesterov, device)
    valid_loss, valid_accuracy = valid_loop(valid_dataloader, model, loss_fn, device)
    loss_train.append(train_loss)
    loss_valid.append(valid_loss)
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# Testing
test_loss, test_accuracy = valid_loop(test_dataloader, model, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 100, Train Loss: 0.0607, Valid Loss: 0.2979, Valid Accuracy: 0.9400
Epoch 200, Train Loss: 0.0123, Valid Loss: 0.3347, Valid Accuracy: 0.9371
Epoch 300, Train Loss: 0.0049, Valid Loss: 0.3799, Valid Accuracy: 0.9400
Epoch 400, Train Loss: 0.0027, Valid Loss: 0.3668, Valid Accuracy: 0.9400
Epoch 500, Train Loss: 0.0019, Valid Loss: 0.3874, Valid Accuracy: 0.9429
Test Loss: 0.4314, Test Accuracy: 0.9187


In [21]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x= np.arange(len(loss_train)), y= loss_train, name= 'Train'))
fig.add_trace(go.Scatter(x= np.arange(len(loss_valid)), y= loss_valid, name= 'Valid'))
fig.show()

### Kết luận: nesterov momentum trong trường hợp này hội tụ lâu hơn và có độ chính xác thấp hơn mometum thông thường

# So sánh giữa các loại optimizer: SGD, RMSPro, và Adam

## RMSProp

In [22]:
# Loss and optimizer
model = NeuralNetWork(input_size=input_size).to(device)
reset_model_weights(model)
lr = 0.01
optimizer_rmsprop = torch.optim.RMSprop(model.parameters(), lr=lr)

# Training
loss_train = []
loss_valid = []

for epoch in range(epochs):
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer_rmsprop, device)
    valid_loss, valid_accuracy = valid_loop(valid_dataloader, model, loss_fn, device)
    loss_train.append(train_loss)
    loss_valid.append(valid_loss)
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# Testing
test_loss, test_accuracy = valid_loop(test_dataloader, model, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 100, Train Loss: 0.0000, Valid Loss: 0.5377, Valid Accuracy: 0.9371
Epoch 200, Train Loss: 0.0000, Valid Loss: 0.7130, Valid Accuracy: 0.9371
Epoch 300, Train Loss: 0.0000, Valid Loss: 0.6786, Valid Accuracy: 0.9371
Epoch 400, Train Loss: 0.0000, Valid Loss: 0.6562, Valid Accuracy: 0.9400
Epoch 500, Train Loss: 0.0000, Valid Loss: 0.6655, Valid Accuracy: 0.9400
Test Loss: 0.7787, Test Accuracy: 0.9227


In [23]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x= np.arange(len(loss_train)), y= loss_train, name= 'Train'))
fig.add_trace(go.Scatter(x= np.arange(len(loss_valid)), y= loss_valid, name= 'Valid'))
fig.show()

## Adam 

In [24]:
model = NeuralNetWork(input_size=input_size).to(device)
reset_model_weights(model)
lr = 0.001
optimizer_adam = torch.optim.Adam(model.parameters(), lr=lr)

In [25]:
# Training
loss_train = []
loss_valid = []

for epoch in range(epochs):
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer_adam, device)
    valid_loss, valid_accuracy = valid_loop(valid_dataloader, model, loss_fn, device)
    loss_train.append(train_loss)
    loss_valid.append(valid_loss)
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# Testing
test_loss, test_accuracy = valid_loop(test_dataloader, model, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch 100, Train Loss: 0.0870, Valid Loss: 0.2893, Valid Accuracy: 0.9171
Epoch 200, Train Loss: 0.0159, Valid Loss: 0.2911, Valid Accuracy: 0.9286
Epoch 300, Train Loss: 0.0028, Valid Loss: 0.3641, Valid Accuracy: 0.9257
Epoch 400, Train Loss: 0.0018, Valid Loss: 0.3578, Valid Accuracy: 0.9229
Epoch 500, Train Loss: 0.0004, Valid Loss: 0.3898, Valid Accuracy: 0.9229
Test Loss: 0.4181, Test Accuracy: 0.9227


In [26]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x= np.arange(len(loss_train)), y= loss_train, name= 'Train'))
fig.add_trace(go.Scatter(x= np.arange(len(loss_valid)), y= loss_valid, name= 'Valid'))
fig.show()

### Kết luận: Adam khi cùng learning rate với SGD và RMSProp thì SGD> Adam > RMSProp 

# Thử các loại Learning Rate Schedules khác nhau và đánh giá

## Step Learning Rate Schedule

In [27]:
model = NeuralNetWork(input_size=input_size).to(device)
reset_model_weights(model)

lr = 0.3
optimizer_sgd = torch.optim.SGD(model.parameters(), lr=lr)
scheduler_step = torch.optim.lr_scheduler.StepLR(optimizer_sgd, step_size=50, gamma=0.1)


In [28]:
loss_train = []
loss_valid = []

for epoch in range(epochs):
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer_sgd, device)  # Pass optimizer, not scheduler
    valid_loss, valid_accuracy = valid_loop(valid_dataloader, model, loss_fn, device)
    
    loss_train.append(train_loss)
    loss_valid.append(valid_loss)
    
    # Step the scheduler at the end of each epoch
    scheduler_step.step()
    
    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# Testing
test_loss, test_accuracy = valid_loop(test_dataloader, model, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 100, Train Loss: 0.5634, Valid Loss: 0.5225, Valid Accuracy: 0.8686
Epoch 200, Train Loss: 0.4621, Valid Loss: 0.5246, Valid Accuracy: 0.8714
Epoch 300, Train Loss: 0.5117, Valid Loss: 0.5101, Valid Accuracy: 0.8714
Epoch 400, Train Loss: 0.4924, Valid Loss: 0.5201, Valid Accuracy: 0.8714
Epoch 500, Train Loss: 0.4978, Valid Loss: 0.5207, Valid Accuracy: 0.8714
Test Loss: 0.5333, Test Accuracy: 0.8420


In [29]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x= np.arange(len(loss_train)), y= loss_train, name= 'Train'))
fig.add_trace(go.Scatter(x= np.arange(len(loss_valid)), y= loss_valid, name= 'Valid'))
fig.show()

## Exponential Learning Rate Schedule

In [30]:
model = NeuralNetWork(input_size=input_size).to(device)
reset_model_weights(model)

lr = 0.3
optimizer_sgd = torch.optim.SGD(model.parameters(), lr=lr)
scheduler_exp = torch.optim.lr_scheduler.ExponentialLR(optimizer_sgd, gamma=0.99)

In [31]:
loss_train = []
loss_valid = []

for epoch in range(epochs):
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer_sgd, device)
    valid_loss, valid_accuracy = valid_loop(valid_dataloader, model, loss_fn, device)
    
    loss_train.append(train_loss)
    loss_valid.append(valid_loss)
    
    # Step the scheduler at the end of each epoch
    scheduler_exp.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_accuracy:.4f}")

# Testing
test_loss, test_accuracy = valid_loop(test_dataloader, model, loss_fn, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


Epoch 100, Train Loss: 0.5412, Valid Loss: 0.5492, Valid Accuracy: 0.8657
Epoch 200, Train Loss: 0.3304, Valid Loss: 0.4217, Valid Accuracy: 0.9029
Epoch 300, Train Loss: 0.3121, Valid Loss: 0.4186, Valid Accuracy: 0.9029
Epoch 400, Train Loss: 0.2787, Valid Loss: 0.3988, Valid Accuracy: 0.9000
Epoch 500, Train Loss: 0.2997, Valid Loss: 0.3919, Valid Accuracy: 0.9000
Test Loss: 0.4283, Test Accuracy: 0.8787


In [32]:
import plotly.graph_objects as go


fig = go.Figure()
fig.add_trace(go.Scatter(x= np.arange(len(loss_train)), y= loss_train, name= 'Train'))
fig.add_trace(go.Scatter(x= np.arange(len(loss_valid)), y= loss_valid, name= 'Valid'))
fig.show()

### Kết luận: Step Learning Rate Schedule hội tụ lâu và có độ chính xác thấp hơn Exponential Learning Rate Schedule